In [2]:
csvlocation = r'C:\Users\yzha2577\Desktop\TorontoNeighborhoodslatLong.csv'
#Getting the Data

In [58]:
#Import Libraries
import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize

In [62]:
import folium
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
#More Libraries

In [11]:
df = pd.read_csv(csvlocation, index_col = 0)
df.head() #Reading CSV and seeing if it looks okay

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [79]:
df.shape

(103, 5)

In [80]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
#Dont worry about teh error Just throw it from your head ;)

c:\users\yzha2577\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [81]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [204]:
#No revealing secrets

In [24]:
#testing for Neighborhood 1
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.8066863, -79.19435340000003.


In [116]:
lim = 105 #There really wasnt that many places
radius = 1000 #1km cuz Toronto is pretty big
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    lim,)

In [117]:
results = requests.get(url).json()
# results

In [118]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [119]:
#Just parsing through the Venues
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
2,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
3,Wendy's,Fast Food Restaurant,43.802008,-79.198080
4,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [120]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

17 venues were returned by Foursquare.


In [121]:
#Self Explanatory Function
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            lim)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [122]:
Toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes = df['Latitude'],
                                   longitudes = df['Longitude'] 
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [123]:
Toronto_venues
Toronto_venues.shape

(2243, 7)

In [124]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",9,9,9,9,9,9
"Alderwood,Long Branch",10,10,10,10,10,10
"Bathurst Manor,Downsview North,Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",23,23,23,23,23,23
Berczy Park,57,57,57,57,57,57


In [125]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 274 uniques categories.


In [126]:
# one hot encoding
TO_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
TO_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [TO_onehot.columns[-1]] + list(TO_onehot.columns[:-1])
TO_onehot = TO_onehot[fixed_columns]

TO_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [128]:
TO_grouped = TO_onehot.groupby('Neighborhood').mean().reset_index()
TO_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Alderwood,Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Bedford Park,Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff,Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [129]:
TO_grouped.shape

(100, 274)

In [130]:
num_top_venues = 5

#Top 5, maybe theres not that many fun stuff
for hood in TO_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = TO_grouped[TO_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Agincourt----
                venue  freq
0        Skating Rink  0.25
1      Sandwich Place  0.25
2              Lounge  0.25
3      Breakfast Spot  0.25
4  Mexican Restaurant  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                venue  freq
0          Playground  0.33
1         Coffee Shop  0.33
2                Park  0.33
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store  0.22
1             Pharmacy  0.11
2           Beer Store  0.11
3       Sandwich Place  0.11
4  Fried Chicken Joint  0.11


----Alderwood,Long Branch----
                venue  freq
0         Pizza Place   0.2


                        venue  freq
0              Massage Studio  0.25
1                        Park  0.25
2  Construction & Landscaping  0.25
3                      Bakery  0.25
4                 Yoga Studio  0.00


----East Birchmount Park,Ionview,Kennedy Park----
               venue  freq
0   Department Store  0.25
1        Bus Station  0.25
2     Discount Store  0.25
3        Coffee Shop  0.25
4  Mobile Phone Shop  0.00


----East Toronto----
                             venue  freq
0                             Park  0.50
1                    Metro Station  0.25
2                Convenience Store  0.25
3                      Yoga Studio  0.00
4  Molecular Gastronomy Restaurant  0.00


----Emery,Humberlea----
                        venue  freq
0  Construction & Landscaping   0.5
1              Baseball Field   0.5
2                 Yoga Studio   0.0
3          Mexican Restaurant   0.0
4         Monument / Landmark   0.0


----Fairview,Henry Farm,Oriole----
                  venu

                             venue  freq
0                           Garden   1.0
1               Mexican Restaurant   0.0
2              Monument / Landmark   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Rouge,Malvern----
                             venue  freq
0             Fast Food Restaurant   1.0
1                    Metro Station   0.0
2  Molecular Gastronomy Restaurant   0.0
3       Modern European Restaurant   0.0
4                Mobile Phone Shop   0.0


----Runnymede,Swansea----
              venue  freq
0              Café  0.10
1       Pizza Place  0.08
2       Coffee Shop  0.08
3  Sushi Restaurant  0.05
4               Gym  0.05


----Ryerson,Garden District----
                       venue  freq
0                Coffee Shop  0.08
1             Clothing Store  0.07
2             Cosmetics Shop  0.04
3                       Café  0.04
4  Middle Eastern Restaurant  0.03


----Scarborough Village----
                          

In [131]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [132]:
num_top_venues = 10
#10 cuz why not
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = TO_grouped['Neighborhood']

for ind in np.arange(TO_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(TO_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(100, 11)

In [172]:
kclusters = 10

#Was boundary testing with 10 tbh

TO_grouped_clustering = TO_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TO_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 5, 0, 0, 0, 0, 0,
       0, 5, 0, 9, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 3, 0, 0, 5, 0, 4, 3, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 3, 5, 0, 5, 0, 0, 0, 5, 0, 0,
       0, 2, 0, 0, 0, 0, 5, 0, 4, 7, 6, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 4, 0, 0, 0, 0, 5, 5])

In [173]:
kmeans.labels_.size

100

In [174]:
TO_merged

,Postcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,Fast Food Restaurant,Women's Store,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,1
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Bar,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,1
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Breakfast Spot,Medical Center,Rental Car Location,Intersection,Mexican Restaurant,Electronics Store,Pizza Place,Spa,Donut Shop,Doner Restaurant,4
3,M1G,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Insurance Office,Korean Restaurant,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,1
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Hakka Restaurant,Fried Chicken Joint,Thai Restaurant,Caribbean Restaurant,Bakery,Bank,Athletics & Sports,Dumpling Restaurant,Drugstore,Donut Shop,1
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,Playground,Convenience Store,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,1
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,Bus Station,Department Store,Coffee Shop,Discount Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Women's Store,1
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,Bakery,Bus Line,Soccer Field,Park,Intersection,Fast Food Restaurant,Metro Station,Dim Sum Restaurant,Diner,Discount Store,1
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,Motel,American Restaurant,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,1
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,College Stadium,Café,Skating Rink,General Entertainment,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,1


In [175]:
TO_merged = df

#Clustering Table Is Added Later Because there are empty columns

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
TO_merged = TO_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

TO_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,Fast Food Restaurant,Women's Store,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Bar,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Breakfast Spot,Medical Center,Rental Car Location,Intersection,Mexican Restaurant,Electronics Store,Pizza Place,Spa,Donut Shop,Doner Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Insurance Office,Korean Restaurant,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Hakka Restaurant,Fried Chicken Joint,Thai Restaurant,Caribbean Restaurant,Bakery,Bank,Athletics & Sports,Dumpling Restaurant,Drugstore,Donut Shop


In [176]:
#Well I guess we gotta drop them since they have no features

TO_merged = TO_merged.drop(16)
TO_merged = TO_merged.drop(21)
TO_merged = TO_merged.drop(93)

In [177]:
# add clustering labels
 TO_merged['Cluster Labels'] = kmeans.labels_

In [178]:
#Forgot to import
import matplotlib.cm as cm
import matplotlib.colors as colors


In [179]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(TO_merged['Latitude'], TO_merged['Longitude'], TO_merged['Neighborhood'], TO_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [181]:
TO_merged.loc[TO_merged['Cluster Labels'] == 0, TO_merged.columns[[1] + list(range(5, TO_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Scarborough,Fast Food Restaurant,Women's Store,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,0
1,Scarborough,Bar,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,0
3,Scarborough,Coffee Shop,Insurance Office,Korean Restaurant,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,0
4,Scarborough,Hakka Restaurant,Fried Chicken Joint,Thai Restaurant,Caribbean Restaurant,Bakery,Bank,Athletics & Sports,Dumpling Restaurant,Drugstore,Donut Shop,0
5,Scarborough,Playground,Convenience Store,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,0
6,Scarborough,Bus Station,Department Store,Coffee Shop,Discount Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Women's Store,0
7,Scarborough,Bakery,Bus Line,Soccer Field,Park,Intersection,Fast Food Restaurant,Metro Station,Dim Sum Restaurant,Diner,Discount Store,0
8,Scarborough,Motel,American Restaurant,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,0
9,Scarborough,College Stadium,Café,Skating Rink,General Entertainment,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,0
10,Scarborough,Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Pet Store,College Stadium,Deli / Bodega,Electronics Store,Eastern European Restaurant,College Gym,0


In [182]:
#Seems Like There is a Clear Majority of East Asian Populated Neighborhoods

In [183]:
TO_merged.loc[TO_merged['Cluster Labels'] == 1, TO_merged.columns[[1] + list(range(5, TO_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
91,Etobicoke,Baseball Field,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,1


In [185]:
#This one is special

In [184]:
TO_merged.loc[TO_merged['Cluster Labels'] == 2, TO_merged.columns[[1] + list(range(5, TO_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
69,Downtown Toronto,Coffee Shop,Restaurant,Café,Hotel,Seafood Restaurant,Cocktail Bar,Beer Bar,Park,Farmers Market,Bakery,2
81,York,Convenience Store,Grocery Store,Bus Line,Caribbean Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,2


In [186]:
#Lots of Restaurants with These two

In [187]:
TO_merged.loc[TO_merged['Cluster Labels'] == 3, TO_merged.columns[[1] + list(range(5, TO_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
38,East York,Sporting Goods Shop,Coffee Shop,Furniture / Home Store,Burger Joint,Beer Store,Fish & Chips Shop,Bagel Shop,Bank,Sports Bar,Dessert Shop,3
44,Central Toronto,Park,Swim School,Construction & Landscaping,Bus Line,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,3
58,Downtown Toronto,Coffee Shop,Café,Thai Restaurant,American Restaurant,Steakhouse,Gym,Bakery,Bar,Hotel,Burger Joint,3


In [188]:
#More Work Out And Cofee 

In [189]:
TO_merged.loc[TO_merged['Cluster Labels'] == 4, TO_merged.columns[[1] + list(range(5, TO_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
43,East Toronto,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Clothing Store,Stationery Store,Bar,Bookstore,Seafood Restaurant,4
76,West Toronto,Supermarket,Bakery,Pharmacy,Middle Eastern Restaurant,Café,Liquor Store,Bank,Bar,Discount Store,Pool,4
82,West Toronto,Café,Mexican Restaurant,Bar,Grocery Store,Furniture / Home Store,Park,Music Venue,Cajun / Creole Restaurant,Diner,Bookstore,4
96,North York,Pizza Place,Empanada Restaurant,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,4


In [190]:
#Lots of Coffee and Baking

In [191]:
TO_merged.loc[TO_merged['Cluster Labels'] == 5, TO_merged.columns[[1] + list(range(5, TO_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
2,Scarborough,Breakfast Spot,Medical Center,Rental Car Location,Intersection,Mexican Restaurant,Electronics Store,Pizza Place,Spa,Donut Shop,Doner Restaurant,5
13,Scarborough,Pizza Place,Rental Car Location,Noodle House,Italian Restaurant,Fried Chicken Joint,Thai Restaurant,Chinese Restaurant,Fast Food Restaurant,Pharmacy,Dog Run,5
17,North York,Athletics & Sports,Dog Run,Golf Course,Fast Food Restaurant,Pool,Mediterranean Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,5
25,North York,Park,Fast Food Restaurant,Food & Drink Shop,Pool,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,5
31,North York,Grocery Store,Bank,Moving Target,Shopping Mall,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Electronics Store,Deli / Bodega,5
41,East Toronto,Greek Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Bookstore,Italian Restaurant,Yoga Studio,Indian Restaurant,Dessert Shop,Bakery,5
59,Downtown Toronto,Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Scenic Lookout,Fried Chicken Joint,Bakery,Restaurant,Brewery,5
61,Downtown Toronto,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Seafood Restaurant,Italian Restaurant,Deli / Bodega,Steakhouse,Bakery,5
65,Central Toronto,Café,Coffee Shop,Sandwich Place,Pizza Place,Flower Shop,History Museum,Jewish Restaurant,Cosmetics Shop,Burger Joint,Indian Restaurant,5
74,York,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy,Gluten-free Restaurant,Gift Shop,Gourmet Shop,Dumpling Restaurant,Drugstore,5


In [192]:
#Dogs, Pizza, And Italy Based Things

In [193]:
TO_merged.loc[TO_merged['Cluster Labels'] == 6, TO_merged.columns[[1] + list(range(5, TO_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
78,West Toronto,Coffee Shop,Café,Breakfast Spot,Performing Arts Venue,Bar,Gym,Furniture / Home Store,Italian Restaurant,Convenience Store,Pet Store,6


In [194]:
#Special

In [195]:
TO_merged.loc[TO_merged['Cluster Labels'] == 7, TO_merged.columns[[1] + list(range(5, TO_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
77,West Toronto,Bar,Asian Restaurant,Coffee Shop,Cocktail Bar,Restaurant,Vietnamese Restaurant,Bakery,Pizza Place,Men's Store,Café,7


In [196]:
#Special

In [197]:
TO_merged.loc[TO_merged['Cluster Labels'] == 8, TO_merged.columns[[1] + list(range(5, TO_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
55,Downtown Toronto,Coffee Shop,Café,Hotel,Restaurant,Cosmetics Shop,Gastropub,Bakery,Italian Restaurant,Breakfast Spot,Pizza Place,8


In [198]:
#Special

In [199]:
TO_merged.loc[TO_merged['Cluster Labels'] == 9, TO_merged.columns[[1] + list(range(5, TO_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
27,North York,Gym,Asian Restaurant,Coffee Shop,Beer Store,Grocery Store,Dim Sum Restaurant,Italian Restaurant,Japanese Restaurant,Discount Store,Sporting Goods Shop,9


In [200]:
#Special

In [205]:
#Saving a copy of the data Again

TO_merged.to_csv("Clustered_Neighborhoods")

In [203]:
# And this Concludes My Findings, Seems liKe there are a lot of East Asian Neighborhoods, and Italian